<a href="https://colab.research.google.com/github/SridharaniKatipally/Cancer-classification/blob/main/cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install biopython tensorflow hyperopt scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.5 MB/s eta 0:00:00


In [6]:
import numpy as np
from Bio import Entrez, SeqIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from hyperopt import fmin, tpe, hp, Trials
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [3]:
Entrez.email = "byula2003@gmail.com"

In [4]:
def fetch_sequences(query, max_sequences, label):
    handle = Entrez.esearch(db="nucleotide", term=query, retmax=max_sequences)
    record = Entrez.read(handle)
    ids = record["IdList"]
    labeled_sequences = []
    for id in ids:
        handle = Entrez.efetch(db="nucleotide", id=id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        labeled_sequences.append((seq_record, label))
    return labeled_sequences

cancer_types = {
    "lung": ("lung cancer AND cancer AND Homo sapiens", "lung AND healthy AND Homo sapiens"),
    "ovarian": ("ovarian cancer AND cancer AND Homo sapiens", "ovarian AND healthy AND Homo sapiens"),
    "colorectal": ("colorectal cancer AND cancer AND Homo sapiens", "colorectal AND healthy AND Homo sapiens"),
    "prostate": ("prostate cancer AND cancer AND Homo sapiens", "prostate AND healthy AND Homo sapiens"),
    "melanoma": ("melanoma AND cancer AND Homo sapiens", "melanoma AND healthy AND Homo sapiens")
}

max_sequences = 100
all_sequences = []

for cancer, (cancer_query, healthy_query) in cancer_types.items():
    cancer_sequences = fetch_sequences(cancer_query, max_sequences, 1)
    healthy_sequences = fetch_sequences(healthy_query, max_sequences, 0)
    all_sequences.extend(cancer_sequences + healthy_sequences)

# Output the labeled sequences
print("Labeled Sequences (first 5 of each cancer type):")
for cancer, (cancer_query, healthy_query) in cancer_types.items():
    cancer_sequences = fetch_sequences(cancer_query, 5, 1)
    healthy_sequences = fetch_sequences(healthy_query, 5, 0)
    print(f"\n{cancer.capitalize()} Cancer Sequences (Label: 1):")
    for seq, label in cancer_sequences:
        print(f"Label: {label}")
        print(seq.format("fasta"))
    print(f"\n{cancer.capitalize()} Healthy Sequences (Label: 0):")
    for seq, label in healthy_sequences:
        print(f"Label: {label}")
        print(seq.format("fasta"))


def augment_sequences(sequences):
    augmented_sequences = []
    for seq, label in sequences:
        augmented_sequences.append((seq, label))
        augmented_sequences.append((seq.reverse_complement(), label))
    return augmented_sequences

all_sequences = augment_sequences(all_sequences)

Labeled Sequences (first 5 of each cancer type):

Lung Cancer Sequences (Label: 1):
Label: 1
>NM_001424868.1 Homo sapiens Fc alpha and mu receptor (FCAMR), transcript variant 4, mRNA
AGGAGGGCGGGATGGAAAATGCCCCTCTTCCTCATACTGTGCCTGCTACAAGGTTCTTCT
TTCGCCCTTCCACAAAAAAGACCCCATCCGAGATGGCTGTGGGAGGGCTCTCTCCCCTCC
AGGACCCATCTCCGGGCCATGGGAACACTCAGGCCTTCCTCGCCCCTCTGCTGGCGGGAG
GAGAGCTCCTTTGCAGCTCCAAATTCATTGAAGGGCTCAAGGCTGGTGTCAGGGGAGCCT
GGAGGAGCTGTCACCATCCAGTGCCATTATGCCCCCTCATCTGTCAACAGGCACCAGAGG
AAGTACTGGTGCCGTCTGGGGCCCCCAAGATGGATCTGCCAGACCATTGTGTCCACCAAC
CAGTATACTCACCATCGCTATCGTGACCGTGTGGCCCTCACAGACTTTCCACAGAGAGGC
TTGTTTGTGGTGAGGCTGTCCCAACTGTCCCCGGATGACATCGGATGCTACCTCTGCGGC
ATTGGAAGTGAAAACAACATGCTGTTCTTAAGCATGAATCTGACCATCTCTGCAGGTCCC
GCCAGCACCCTCCCCACAGCCACTCCAGCTGCTGGGGAGCTCACCATGAGATCCTATGGA
ACAGCGTCTCCAGTGGCCAACAGATGGACCCCAGGAACCACCCAGACCTTAGGACAGGGG
ACAGCATGGGACACAGTTGCTTCCACTCCAGGAACCAGCAAGACTACAGCTTCAGCTGAG
GGAAGACGAACCCCAGGAGCAACCAGGCCAGCAGCTCCAGGGACAGGCAGCTGGGCAGAG
GGTTCTGTCAAAGCACCTGCTCCG

In [6]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

def preprocess_sequences(sequences):
    kmers = [' '.join([str(sequence.seq[i:i+3]) for i in range(len(sequence.seq) - 2)]) for sequence, label in sequences]
    labels = [label for sequence, label in sequences]

    # Print the first few k-mers for inspection
    print("Sample k-mers:")
    for i in range(min(5, len(kmers))):  # Print up to 5 k-mers
        print(f"Sequence {i+1}: {kmers[i]}")

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(kmers).toarray()  # Convert sparse matrix to dense matrix
    y = np.array(labels)  # Convert labels to numpy array

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Preprocess the collected sequences
X, y = preprocess_sequences(all_sequences)

print(f"Shape of X: {X.shape}")
print(f"Number of labels: {len(y)}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

def create_mlp_model(input_shape):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

mlp_model = create_mlp_model((X_train.shape[1],))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

mlp_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[early_stopping, reduce_lr])

def evaluate_model(model, X_test, y_test):
    pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, (pred_test > 0.5).astype(int))
    precision = precision_score(y_test, (pred_test > 0.5).astype(int))
    recall = recall_score(y_test, (pred_test > 0.5).astype(int))
    f1 = f1_score(y_test, (pred_test > 0.5).astype(int))
    auc = roc_auc_score(y_test, pred_test)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")

# Evaluate the MLP model
evaluate_model(mlp_model, X_test, y_test)


Sample k-mers:
Sequence 1: AGG GGA GAG AGG GGG GGC GCG CGG GGG GGA GAT ATG TGG GGA GAA AAA AAA AAT ATG TGC GCC CCC CCC CCT CTC TCT CTT TTC TCC CCT CTC TCA CAT ATA TAC ACT CTG TGT GTG TGC GCC CCT CTG TGC GCT CTA TAC ACA CAA AAG AGG GGT GTT TTC TCT CTT TTC TCT CTT TTT TTC TCG CGC GCC CCC CCT CTT TTC TCC CCA CAC ACA CAA AAA AAA AAA AAA AAG AGA GAC ACC CCC CCC CCA CAT ATC TCC CCG CGA GAG AGA GAT ATG TGG GGC GCT CTG TGT GTG TGG GGG GGA GAG AGG GGG GGC GCT CTC TCT CTC TCT CTC TCC CCC CCC CCT CTC TCC CCA CAG AGG GGA GAC ACC CCC CCA CAT ATC TCT CTC TCC CCG CGG GGG GGC GCC CCA CAT ATG TGG GGG GGA GAA AAC ACA CAC ACT CTC TCA CAG AGG GGC GCC CCT CTT TTC TCC CCT CTC TCG CGC GCC CCC CCC CCT CTC TCT CTG TGC GCT CTG TGG GGC GCG CGG GGG GGA GAG AGG GGA GAG AGA GAG AGC GCT CTC TCC CCT CTT TTT TTG TGC GCA CAG AGC GCT CTC TCC CCA CAA AAA AAT ATT TTC TCA CAT ATT TTG TGA GAA AAG AGG GGG GGC GCT CTC TCA CAA AAG AGG GGC GCT CTG TGG GGT GTG TGT GTC TCA CAG AGG GGG GGG GGA GAG AGC GCC CCT CTG TGG GGA GAG AGG G

In [9]:
#LSTM
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def preprocess_sequences(sequences):
    kmers = [' '.join([str(sequence.seq[i:i+3]) for i in range(len(sequence.seq) - 2)]) for sequence, label in sequences]
    labels = [label for sequence, label in sequences]

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(kmers).toarray()
    y = np.array(labels)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Preprocess the collected sequences
X, y = preprocess_sequences(all_sequences)

# Reshape the data for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(64)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = create_cnn_lstm_model((X_train.shape[1], X_train.shape[2]))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

cnn_lstm_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
                   callbacks=[early_stopping, reduce_lr])

def evaluate_model(model, X_test, y_test):
    pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, (pred_test > 0.5).astype(int))
    precision = precision_score(y_test, (pred_test > 0.5).astype(int))
    recall = recall_score(y_test, (pred_test > 0.5).astype(int))
    f1 = f1_score(y_test, (pred_test > 0.5).astype(int))
    auc = roc_auc_score(y_test, pred_test)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")

# Evaluate the CNN-LSTM model
evaluate_model(cnn_lstm_model, X_test, y_test)


Epoch 1/100
46/46 [==============================] - 21s 235ms/step - loss: 0.7630 - accuracy: 0.5923 - val_loss: 0.6859 - val_accuracy: 0.5820 - lr: 0.0010
Epoch 2/100
46/46 [==============================] - 7s 157ms/step - loss: 0.6830 - accuracy: 0.6204 - val_loss: 0.6776 - val_accuracy: 0.6175 - lr: 0.0010
Epoch 3/100
46/46 [==============================] - 11s 238ms/step - loss: 0.6791 - accuracy: 0.6306 - val_loss: 0.6530 - val_accuracy: 0.5464 - lr: 0.0010
Epoch 4/100
46/46 [==============================] - 9s 196ms/step - loss: 0.6269 - accuracy: 0.6683 - val_loss: 0.6462 - val_accuracy: 0.5574 - lr: 0.0010
Epoch 5/100
46/46 [==============================] - 7s 157ms/step - loss: 0.6278 - accuracy: 0.6648 - val_loss: 0.6560 - val_accuracy: 0.5984 - lr: 0.0010
Epoch 6/100
46/46 [==============================] - 9s 201ms/step - loss: 0.6030 - accuracy: 0.6792 - val_loss: 0.6342 - val_accuracy: 0.6339 - lr: 0.0010
Epoch 7/100
46/46 [==============================] - 8s 176ms/

In [10]:
#GRU
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Bidirectional, GRU, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def preprocess_sequences(sequences):
    kmers = [' '.join([str(sequence.seq[i:i+3]) for i in range(len(sequence.seq) - 2)]) for sequence, label in sequences]
    labels = [label for sequence, label in sequences]

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(kmers).toarray()
    y = np.array(labels)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Preprocess the collected sequences
X, y = preprocess_sequences(all_sequences)

# Reshape the data for GRU
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Bidirectional(GRU(64)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

gru_model = create_gru_model((X_train.shape[1], X_train.shape[2]))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

gru_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[early_stopping, reduce_lr])

def evaluate_model(model, X_test, y_test):
    pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, (pred_test > 0.5).astype(int))
    precision = precision_score(y_test, (pred_test > 0.5).astype(int))
    recall = recall_score(y_test, (pred_test > 0.5).astype(int))
    f1 = f1_score(y_test, (pred_test > 0.5).astype(int))
    auc = roc_auc_score(y_test, pred_test)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")

# Evaluate the GRU model
evaluate_model(gru_model, X_test, y_test)


Epoch 1/100
46/46 [==============================] - 19s 197ms/step - loss: 0.8262 - accuracy: 0.5609 - val_loss: 0.6692 - val_accuracy: 0.6148 - lr: 0.0010
Epoch 2/100
46/46 [==============================] - 7s 145ms/step - loss: 0.6849 - accuracy: 0.6347 - val_loss: 0.6629 - val_accuracy: 0.6011 - lr: 0.0010
Epoch 3/100
46/46 [==============================] - 6s 120ms/step - loss: 0.6759 - accuracy: 0.6218 - val_loss: 0.6901 - val_accuracy: 0.5137 - lr: 0.0010
Epoch 4/100
46/46 [==============================] - 6s 123ms/step - loss: 0.6190 - accuracy: 0.6689 - val_loss: 0.7022 - val_accuracy: 0.4891 - lr: 0.0010
Epoch 5/100
46/46 [==============================] - 7s 146ms/step - loss: 0.5923 - accuracy: 0.6908 - val_loss: 0.6689 - val_accuracy: 0.5383 - lr: 0.0010
Epoch 6/100
46/46 [==============================] - 5s 116ms/step - loss: 0.5835 - accuracy: 0.6997 - val_loss: 0.6430 - val_accuracy: 0.6311 - lr: 0.0010
Epoch 7/100
46/46 [==============================] - 10s 212ms/

In [11]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Bidirectional, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def preprocess_sequences(sequences):
    kmers = [' '.join([str(sequence.seq[i:i+3]) for i in range(len(sequence.seq) - 2)]) for sequence, label in sequences]
    labels = [label for sequence, label in sequences]

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(kmers).toarray()
    y = np.array(labels)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Preprocess the collected sequences
X, y = preprocess_sequences(all_sequences)

# Reshape the data for Bidirectional LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

def create_bidirectional_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(64)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

bidirectional_lstm_model = create_bidirectional_lstm_model((X_train.shape[1], X_train.shape[2]))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

bidirectional_lstm_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
                             callbacks=[early_stopping, reduce_lr])

def evaluate_model(model, X_test, y_test):
    pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, (pred_test > 0.5).astype(int))
    precision = precision_score(y_test, (pred_test > 0.5).astype(int))
    recall = recall_score(y_test, (pred_test > 0.5).astype(int))
    f1 = f1_score(y_test, (pred_test > 0.5).astype(int))
    auc = roc_auc_score(y_test, pred_test)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")

# Evaluate the Bidirectional LSTM model
evaluate_model(bidirectional_lstm_model, X_test, y_test)


Epoch 1/100
46/46 [==============================] - 20s 231ms/step - loss: 0.7748 - accuracy: 0.5862 - val_loss: 0.6658 - val_accuracy: 0.6175 - lr: 0.0010
Epoch 2/100
46/46 [==============================] - 9s 196ms/step - loss: 0.6892 - accuracy: 0.6368 - val_loss: 0.6325 - val_accuracy: 0.6202 - lr: 0.0010
Epoch 3/100
46/46 [==============================] - 7s 156ms/step - loss: 0.6803 - accuracy: 0.6341 - val_loss: 0.6532 - val_accuracy: 0.6038 - lr: 0.0010
Epoch 4/100
46/46 [==============================] - 9s 198ms/step - loss: 0.6370 - accuracy: 0.6607 - val_loss: 0.6589 - val_accuracy: 0.6038 - lr: 0.0010
Epoch 5/100
46/46 [==============================] - 7s 150ms/step - loss: 0.6478 - accuracy: 0.6621 - val_loss: 0.6073 - val_accuracy: 0.6148 - lr: 0.0010
Epoch 6/100
46/46 [==============================] - 9s 198ms/step - loss: 0.6187 - accuracy: 0.6676 - val_loss: 0.6035 - val_accuracy: 0.6366 - lr: 0.0010
Epoch 7/100
46/46 [==============================] - 8s 176ms/s